In [16]:
#hide
#! [ -e /content ]

#hide
#This imports and sets up everything you will need for this notebook
#
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random

from fastai.imports import *
np.set_printoptions(linewidth=130)

# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path
import os
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import xgboost as xgb


from ipywidgets import interact


matplotlib.rc('image', cmap='Greys')

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/playground-series-s4e4/sample_submission.csv
/kaggle/input/playground-series-s4e4/train.csv
/kaggle/input/playground-series-s4e4/test.csv


In [18]:
!ls /kaggle/input/playground-series-s4e4

sample_submission.csv  test.csv  train.csv


Set random seed for reproducibility.

In [19]:
#set_seed(42)

In [20]:
path = Path('/kaggle/input/playground-series-s4e4/')
path

Path('/kaggle/input/playground-series-s4e4')

Index_col

With or without.

I remove the index_col because it changes the shape of our original dataset.
 Later on we shall be able to investigate the effect of this action.

In [21]:
#train_df = pd.read_csv(path/'train.csv',index_col='id')

train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv',index_col='id')


In [22]:
train_df.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,,
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [23]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='Rings')

In [24]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [25]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Rings',
                   y_block=RegressionBlock(),
                   splits=splits)

In [26]:
to.xs.iloc[:2]

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
id,,,,,,,,
67483,3,-0.354354,-0.473744,-0.141812,-0.692268,-0.687521,-0.766082,-0.585808
79859,1,0.828596,1.054672,0.780734,0.600354,0.225346,0.765353,1.010391


In [27]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

Random Forests

In [28]:
rf = RandomForestClassifier(100, min_samples_leaf=5)
rf.fit(X_train, y_train);
#mean_absolute_error(y_test, rf.predict(X_test))

#roc_auc_score(y_test, rf.predict(X_test))

In [29]:
rf_preds = rf.predict(X_test)

In [31]:
from sklearn.metrics import mean_squared_error

# Assuming y_true is your array of true values and y_pred is your array of predicted values
rmse = mean_squared_error(y_test, rf_preds, squared=False)


In [32]:
rmse

2.032471054240814

In [ ]:
#rmse(rf_preds, tensor(y_test))

Kaggle submission code from


https://www.kaggle.com/code/nguyncaoduy/fastai-tabular-regression-model-nn-xgb



In [33]:
dls = to.dataloaders(bs=64)

In [34]:
test_dl = dls.test_dl(test_df)

In [35]:
preds = tensor(rf.predict(test_dl.xs))
rf_preds = preds

In [36]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Rings'] = rf_preds # nn_preds
submit.head()

,id,Rings
0,90615,10
1,90616,9
2,90617,9
3,90618,12
4,90619,8


In [37]:
submit.to_csv('submission.csv', index=False)

In [38]:
!ls

submission.csv


In [39]:
sub = pd.read_csv('submission.csv')
sub.head()

,id,Rings
0,90615,10
1,90616,9
2,90617,9
3,90618,12
4,90619,8


In [25]:
!kaggle competitions submit -c playground-series-s4e4 -f submission.csv -m "random forest initial subm trial 1 with xgboost sub code"

Traceback (most recent call last):

  File "/usr/local/bin/kaggle", line 8, in <module>

    sys.exit(main())

  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 70, in main

    out = args.func(**command_args)

  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 801, in competition_submit_cli

    submit_result = self.competition_submit(file_name, message,

  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 771, in competition_submit

    url_result['createUrl'],

KeyError: 'createUrl'


sub 2

In [ ]:
#target_preds = ens_preds[0]
#rf_target_preds = rf_preds[0]
rf_preds = rf.predict(X_test)
rf_target_preds = rf_preds
test_df['Rings'] = rf_target_preds

test_df.to_csv('submission.csv', columns=['Rings'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

Sub method 3

In [ ]:
def subm(preds, suff):
    test_df['Rings'] = preds
    sub_df = test_df[['id','Rings']]
    sub_df.to_csv(f'sub{suff}.csv', index=False)

subm(ens_preds, 'mission')

In [ ]:
#sub_df = pd.DataFrame(ens_preds, columns='Rings')
#sub_df['id'] = test_df['id']

# Reorder the columns to match the submission format
#sub_df = sub_df[['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

# Save the submission DataFrame as a CSV file
#sub_df.to_csv('rf_submission.csv', index=False)

In [ ]:
#targets ='Rings'
#test_df[targets] = ens_preds
#sub_df['id'] = test_df['id']

#test_df.to_csv('submission.csv', columns=['Rings'], index=False)

#ens_sub = pd.read_csv('submission.csv')
#ens_sub.head()

,Rings
0,9.475194
1,9.920687
2,10.470133
3,10.523545
4,7.595455
